In [14]:
#JUPYTER NOTEBOOK INTERFACE TO SEC FILING DATABASE

#The first part of this notebook provides a client to the Django database as a REST API. The client allows for
#the searching of the tables of financial data contained in a SEC filing

#The second part of this notebook provides a parser that processes and cleans the found table data into a more useful format



###########################################  PART 1 ######################################################

#The API provides 3 endpoints with the following purposes: 
#(i) getting a list of companies in database; 
#(ii) getting a list of filings for a certain company in the database by CIK (unique id number of a company used by the SEC), and 
#(iii) searching the tables of financial data in a filing; search can be performed for a financial statement type
# or for certain terms

import pandas
import numpy
import requests
import json

#set pandas display options to improve visibility of data
pandas.set_option("display.max_colwidth", -1)
pandas.set_option("display.width", 1000)
pandas.set_option("display.max_rows", 1000)
pandas.set_option("display.max_columns", 1000)

class DatabaseAPIClient():
    
    BASE_API_URL = "http://openedgar:8000/api/companies/"
    
    #list all companies in database
    #returns: dict w/ keys "date", "cik", "name", "sic"
    def get_companies(self):
        response = requests.get(self.BASE_API_URL).text
        return json.loads(response)
    
    #list all filings for a single company
    #returns Dict with keys: "id", "accession_number", "date_filed", "form_type", "is_error", "is_processed", "notes"    
    def get_filings(self, cik):
        response = requests.get(self.BASE_API_URL + cik).text
        return json.loads(response)

    #search tables in a single filing
    #key parameter: the json_request_data argument identifies the type of search to perform through the key "request_type"
    #returns: 
    def search_single_filing(self, cik, filing_pk, json_request_data):
        response = requests.get("{}{}/filing/{}".format(self.BASE_API_URL, cik, filing_pk), json=json_request_data).text
        if response:
            return json.loads(response)
        else:
            return None
        
    #displays results of get_companies
    def display_companies(self):
        companies = self.get_companies()
        display(pandas.DataFrame(companies))

    #displays results of get_filings
    def display_company_filings(self, cik):
        filings = self.get_filings(cik)
        df = pandas.DataFrame(filings)
        df.index = df["id"]
        display(df)

    #iterates over all of a company's filings and displays search results
    def display_all_search_results(self, cik, json_request_data):
        filings = self.get_filings(cik)
        for result_index, filing in enumerate(filings):
            print("id: {0}; accession number: {1}".format(filing['id'], filing['accession_number']))
            response = self.search_single_filing(cik, filing["id"], json_request_data)
            if response:
                for r in response['results']:
                    display(pandas.DataFrame(r['table_data']))
            else:
                print("NONE FOUND")

    def search_results(self, cik, json_request_data):
        results = []
        filings = self.get_filings(cik)
        for result_index, filing in enumerate(filings):
            print("processing id: {0}; accession number: {1}".format(filing['id'], filing['accession_number']))
            response = self.search_single_filing(cik, filing["id"], json_request_data)
            if response:
                for r in response['results']:
                    results.append(r['table_data'])
        return results
        


In [15]:
#this cell can be used to execute commands

client = DatabaseAPIClient()

#cik - the CIK identification number for the company to be searched in database
cik = "910329"

#filing_pk - the primary key for the filing to be searched in database
filing_pk = 590

#json_request_data - sets forth options for the searching of tables in filings
json_request_data = {
    #type of table to be searched in filing
    "request_type": "financial_statements",
    
    #if request type is financial statement, type of financial statements requested
    "statement_type": "balance_sheet",

    #if request tyep is search, terms to search in tables of filing
    "search_terms": ["term1", "term2", "term3"],
}


#client.display_companies()
#client.display_company_filings(cik)
#client.display_all_search_results(cik, json_request_data)


In [16]:
########################################### PART 2 ###########################################

# As can be seen from a set of table search results, data obtained directly from a HTML or plain text
# table of a SEC filing is not consistently organized and can vary between different companies and time frames
# one significant issue in parsing HTML tables contained in SEC filings is the prevelance of "colspan" and "rowspan"
# attributes; the HTML table parser (set forth in the Django codebase) duplicates the cell content across all cells
# covered by a colspan or row span. That duplication needs to be addressed here

#This cell provides the class used to parse the table search results

import itertools

#parser
class SECFilingTableParser():
    
    #primary parse method
    def parse(self, data_array):
        #preliminary processing dataframe; see description in method below
        df = self.clean_df(pandas.DataFrame(data_array))
        
        #if the dataframe is only 1 row or 1 column, we're not interested in parsing any further
        if len(df.index) == 1 or len(df.columns) == 1:
            return df
        else:
            try:
                #this parser assumes the first row in the first column of a table is the first row of data

                #find the first column
                first_column = df.iloc[:,0].astype('object')
                #find the first row that contains either letters or numbers in the first column
                row_start = first_column.str.contains("[A-Za-z0-9]", case=False, regex=True, na=False).idxmax()
                
                #if text is contained in position [0] then assume that the table only contains columns (and no index for rows)
                if row_start == 0:
                    return self.all_columns_table(df, row_start)
                #else, parse as if there is an index for the rows
                else:
                    return self.accounts_table(df, row_start)
            except:
                print("ERROR")
                print("Dataframe:")
                display(pandas.DataFrame(data_array))
                print("array")
                display(data_array)
    
    def all_columns_table(self, df, row_start):
        #obtain text in the first row, assume that they are the heading for each column
        columns = df.iloc[row_start,:]
        #separate data from headings
        df = df.loc[row_start+1:,:]
        #set the new column headings
        df.columns = columns
        
        #more cleanup
        df = df.replace('^\s*$', numpy.nan, regex=True, inplace=False)
        #assumed that there are duplicate columns headings; saving a final set of column names to be used later
        final_columns = df.columns.unique().dropna()
        
        #to elminate the duplicate cell content without losing any other unique data, the data is groupedby
        #the columns and then the process_account_name function concatenates all unique data while ignoring
        #duplicates
        new_df = {}
        for i,g in df.groupby(df.columns, axis=1):
            new_df[i] = g.apply(lambda x: self.process_account_name(x), axis=1).values.tolist()
        new_df = pandas.DataFrame(new_df)
        new_df = new_df[final_columns]

        #more clean up
        new_df.dropna(how="all", axis=1, inplace=True)
        new_df.dropna(how="all", axis=0, inplace=True)
        return new_df        

    def clean_dollar_amounts(self, df):
        #this function eliminates unnecessary markings usually found in these tables
        df.replace('[$]', "", regex=True, inplace=True)
        df.replace('[)]', "", regex=True, inplace=True)
        df.replace('[(]', "-", regex=True, inplace=True)
        df.replace('[,]', "", regex=True, inplace=True)
        df = df.apply(lambda x: x.str.strip(), axis=1)
        return df
                    
    def accounts_table(self, df, row_start):
        #for these types of tables, assume that first row is the start of the row index
        first_row = df.iloc[row_start-1,:].astype('object')
        #assume that the next cell of text in our "first row" is the first column of data
        col_start = first_row.astype('object').str.contains("[A-Za-z0-9]", case=False, regex=True, na=False).idxmax()

        #assume that all columns before "col_start" is a part of the row index
        index_columns = df.iloc[row_start:, :col_start]
        #use process_account_name method to eliminate duplicates but concatenate unique text
        index = index_columns.apply(lambda x: self.process_account_name(x), axis=1)

        #assume that all rows above "row_start" and columns after "col_start" pertain to the column headings
        column_header_rows = df.iloc[:row_start, col_start:]
        #use process_account name to elminate duplicates and concatenate unique text
        columns = column_header_rows.apply(lambda x: self.process_account_name(x))
        
        #separate data from row index and column headings
        df = df.loc[row_start:, col_start:]
        #eliminate unncessary markings usually found in numeric data
        df = self.clean_dollar_amounts(df)
        #clean up spaces
        df = df.replace('^\s*$', numpy.nan, regex=True, inplace=False)
        
        #add column headings to data
        df.columns = columns
        #add row index to data
        df.insert(0, "account", index)
        #clean up duplicate or blank columns
        final_columns = df.columns.unique().dropna()
        blank_columns = final_columns.str.contains("^\s*$", regex=True, na=True)
        final_columns = final_columns[~blank_columns]

        #to elminate the duplicate cell content without losing any other unique data, the data is groupedby
        #the columns and then the process_account_name function concatenates all unique data while ignoring
        #duplicates
        new_df = {}
        for i,g in df.groupby(df.columns, axis=1):
            new_df[i] = g.apply(lambda x: self.process_account_name(x), axis=1).values.tolist()
        new_df = pandas.DataFrame(new_df)
        new_df = new_df[final_columns]
        
        #more clean up
        new_df.dropna(how="all", axis=1, inplace=True)
        new_df.dropna(how="all", axis=0, inplace=True)
        return new_df

    def process_account_name(self, old_series):
        #method to process related adjacent cells to elminate duplicates but keep unique text
        series = old_series.dropna()
        return " ".join([i for i,g in itertools.groupby(series, key=lambda x: x)])
    
    def clean_df(self, df):
        #preliminary processing of data
        new_df = df
        #remove extraneous weird spaces/characters
        new_df = new_df.replace(self.extraneous_chars(), "", regex=True, inplace=False)
        #replace all blank cells with nan
        new_df = new_df.replace('^\s*$', numpy.nan, regex=True, inplace=False)
        #replace all cells that have the string "None" in it
        new_df = new_df.replace("^\s*None\s*$", numpy.nan, regex=True, inplace=False)
        #replace all cells that have the None type object with nan
        new_df = new_df.fillna(numpy.nan)
        #replace words commonly found in the first row in the first column but that is not the
        #start of real data
        new_df = new_df.replace(self.blacklist_repeating_headers(), numpy.nan, regex=True, inplace=False)
        #process to eliminate all blank rows and columns
        nonempty_rows = new_df.nunique(axis=1, dropna=False) <= 1 #df.apply(lambda x: is_all_same(x), axis=1)
        nonempty_columns = new_df.nunique(axis=0, dropna=False) <= 1 #df.apply(lambda x: is_all_same(x))
        new_df = new_df.loc[~nonempty_rows, ~nonempty_columns]
        #set new numbered indexes and columns 
        new_df.index = range(new_df.shape[0])
        new_df.columns = range(new_df.shape[1])
        return new_df

    def blacklist_repeating_headers(self):
        return [
            "^\s*ANALYSIS OF LOSS AND LOSS ADJUST",
            "^\s*RECONCILIATION OF NET RESERVES FOR LOSSES",
            "^\s*[(].*[Tt][Hh][Oo][Uu][Ss][Aa][Nn][Dd].*[)]\s*$",
            "^\s*[(].*[Mm][Ii][Ll][Ll][Ii][Oo][Nn].*[)]\s*$",
            "^\s*[Yy][Ee][Aa][Rr][.]*\s*[Ee][Nn][Dd][Ee][Dd].*$",
            "^\s*[Ff][Oo][Rr]\s*[Tt][Hh][Ee]\s*[Yy][Ee][Aa][Rr][.]*\s*[Ee][Nn][Dd][Ee][Dd].*$",
            "^\s*CONSOLIDATED STATEMENTS OF INCOME\s*$",
            "For the years ended",
            "FOR THE YEARS ENDED",
            "[(]millio",
            "ns-except per sh",
            "are amounts[)]",
            "^\s*December 31,\s*$",
            "^\s*ecember 31,\s*$",
            "^\s*DECEMBER 31,\s*$",
            "Years Ended December 31"
        ]
    
    def extraneous_chars(self):
        return [
            '\xa0',
            '\xA0',
            '\n',
        ]


In [19]:
#this cell can be used to execute the parsing of the table search results

#table search results using parameters set in prior part
client = DatabaseAPIClient()
tables = client.search_results(cik, json_request_data)

parser = SECFilingTableParser()

for table in tables:
    print("*** table prior to parsing ***")
    display(pandas.DataFrame(table))
    print("*** table after parsing ***")
    display(parser.parse(table))


processing id: 1; accession number: 0001558370-21-001985
processing id: 2; accession number: 0001558370-20-001889
processing id: 3; accession number: 0001144204-19-011678
processing id: 4; accession number: 0001144204-18-015288
processing id: 5; accession number: 0001144204-17-015143
processing id: 6; accession number: 0001144204-16-088200
processing id: 7; accession number: 0001144204-15-026672
processing id: 8; accession number: 0001144204-15-016333
processing id: 9; accession number: 0001144204-14-016118
processing id: 10; accession number: 0001144204-13-015443
processing id: 11; accession number: 0001144204-12-015310
processing id: 12; accession number: 0001144204-11-019422
processing id: 13; accession number: 0001144204-11-001968
processing id: 14; accession number: 0001144204-10-016775
processing id: 15; accession number: 0001144204-09-014237
processing id: 16; accession number: 0001144204-08-015754
processing id: 17; accession number: 0001144204-07-058045
processing id: 18; acce

,0,1,2,3,4,5,6,7
0,​,​,​,​,​,​,​,​
1,​,​,​,​,2020,​,​,2019
2,​,​,​,​,​,​,​,​
3,ASSETS,ASSETS,ASSETS,ASSETS,ASSETS,ASSETS,ASSETS,ASSETS
4,Current Assets,​,​,​,​,​,​,​
5,Cash and cash equivalents,​,​,$,"163,723",​,$,"76,974"
6,Accounts receivable - net of doubtful accounts of $219 and $235 at,​,​,​,​,​,​,​
7,"December 31, 2020 and 2019, respectively",​,​,​,584,​,​,"1,437"
8,Inventories,​,​,​,"53,392",​,​,"48,771"
9,Investment securities,​,​,​,"10,752",​,​,"15,704"


*** table after parsing ***


,account,2020,​,2019
0,Current Assets ​,​,​,​
1,Cash and cash equivalents ​ $,163723,​,76974
2,Accounts receivable - net of doubtful accounts of $219 and $235 at ​,​,​,​
3,"December 31, 2020 and 2019, respectively ​",584,​,1437
4,Inventories ​,53392,​,48771
5,Investment securities ​,10752,​,15704
6,"Income taxes, prepaid ​",-,​,5169
7,Prepaid expenses and other current assets ​,5863,​,6096
8,Total current assets ​,234314,​,154151
9,"Property, plant and equipment - net of accumulated depreciation ​",27633,​,26039


*** table prior to parsing ***


...

,account,2006,2005
0,ASSETS,,
1,Current assets:,,
2,Cash and cash equivalents,1085000,1484000
3,Accounts receivable-net of allowance for doubtful accounts,448000,985000
4,"of $100,000",,
5,Inventory,8255000,5475000
6,Investment securities,1540000,2700000
7,Deferred compensation,673000,525000
8,Prepaid expenses and other current assets,2599000,3273000
9,Note receivable - current,174000,


*** table prior to parsing ***


,0,1,2
0,,2005,2004
1,ASSETS,,
2,CURRENT ASSETS:,,
3,Cash,"$ 1,484,000","$ 612,000"
4,Accounts receivable-net of allowance for doubtful accounts,"985,000","1,063,000"
5,"of $100,000 and $87,000",,
6,Inventory,"5,475,000","4,251,000"
7,Investment securities,"2,700,000","2,626,000"
8,Deferred compensation,"525,000","321,000"
9,Prepaid expenses and other current assets,"3,273,000","1,079,000"


*** table after parsing ***


,account,2005,2004
0,ASSETS,,
1,CURRENT ASSETS:,,
2,Cash,1484000,612000
3,Accounts receivable-net of allowance for doubtful accounts,985000,1063000
4,"of $100,000 and $87,000",,
5,Inventory,5475000,4251000
6,Investment securities,2700000,2626000
7,Deferred compensation,525000,321000
8,Prepaid expenses and other current assets,3273000,1079000
9,Current portion of deferred tax asset,-,19000
